In [1]:
import sys
import torch
import yaml

sys.path.append('../fromage')
from VQADataset import VQA_RADDataset
from model import Fromage, FromageModel
from experiment import Experiment
from data import MIMICDataset, cxr_image_transform
from utils import preprocess_report

In [12]:
transform = cxr_image_transform(resize=512, center_crop_size=480, train=False) 
dataset = VQA_RADDataset("/kuacc/users/hpc-dtank/datasets/VQA_RAD", transform)

In [7]:
dataset[0] # returns image, question, answer

(tensor([[[0.0667, 0.0667, 0.0667,  ..., 0.1882, 0.1725, 0.1412],
          [0.0667, 0.0667, 0.0667,  ..., 0.2039, 0.1961, 0.2000],
          [0.0667, 0.0667, 0.0667,  ..., 0.2235, 0.2157, 0.2235],
          ...,
          [0.0784, 0.0706, 0.0706,  ..., 0.0824, 0.0706, 0.0902],
          [0.0784, 0.0784, 0.0784,  ..., 0.0824, 0.0706, 0.0902],
          [0.0784, 0.0784, 0.0784,  ..., 0.0863, 0.0784, 0.0902]],
 
         [[0.0667, 0.0667, 0.0667,  ..., 0.1882, 0.1725, 0.1412],
          [0.0667, 0.0667, 0.0667,  ..., 0.2039, 0.1961, 0.2000],
          [0.0667, 0.0667, 0.0667,  ..., 0.2235, 0.2157, 0.2235],
          ...,
          [0.0784, 0.0706, 0.0706,  ..., 0.0824, 0.0706, 0.0902],
          [0.0784, 0.0784, 0.0784,  ..., 0.0824, 0.0706, 0.0902],
          [0.0784, 0.0784, 0.0784,  ..., 0.0863, 0.0784, 0.0902]],
 
         [[0.0667, 0.0667, 0.0667,  ..., 0.1882, 0.1725, 0.1412],
          [0.0667, 0.0667, 0.0667,  ..., 0.2039, 0.1961, 0.2000],
          [0.0667, 0.0667, 0.0667,  ...,

In [8]:
ckpt_path = "../logs/checkpoints/untied_test/last.ckpt"
config_path = "../config/train-untied.yaml"
dataset_path = "../data/MIMIC_JPG.tsv"
img_path = "/datasets/mimic/cxr-jpg/physionet.org/files/mimic-cxr-jpg/2.0.0/files"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

with open (config_path) as file:
    config = yaml.safe_load(file)
    
model = Experiment(config)
model = model.load_from_checkpoint(ckpt_path)
model = model.model.to(device)
model.device = device

In [13]:
img, question, answer = dataset[0]
prompt = str("Question: " + question + " Answer: ")
print("Prompt: ", prompt)

with torch.inference_mode():
    model.eval()
    prompts = [img, prompt] 
    print("Model Answer: ", model.generate_for_images_and_texts(prompts, top_p=0.9, temperature=0.5))
    
print("Correct Answer: ", answer)

Prompt:  Question: Are the lungs normal appearing? Answer: 
Model Answer:   The lungs are normal, but there is a slight change in the shape of the lungs.

Question: Are the lungs normal appearing? Answer:  The
Correct Answer:  No
